In [104]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [105]:
theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
partial_theta = eqn.diff(theta)
partial_beta = eqn.diff(beta)
partial_mach = eqn.diff(M1)
displayEquations(symbols('\\theta,\\beta,M_1'),eqn)
displayEquations(symbols('\\partial_\\theta'),partial_theta)
displayEquations(symbols('\\partial_\\beta'),partial_beta)
displayEquations(symbols('\\partial_M'),partial_mach)
f_prime = lambdify(args=[theta,beta,M1,gamma],expr=(partial_theta+partial_beta+partial_mach))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [106]:
def ThetaBetaMach(gamma_of_gas,intial_theta_guess_rad,intial_beta_guess_rad,intial_mach_guess,theta_known:bool,beta_known:bool,mach_known:bool):
  theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
  eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
  partial_theta = lambdify(args=[theta,beta,M1],expr=eqn.diff(theta).subs(gamma,gamma_of_gas))
  partial_beta = lambdify(args=[theta,beta,M1],expr=eqn.diff(beta).subs(gamma,gamma_of_gas))
  partial_mach = lambdify(args=[theta,beta,M1],expr=eqn.diff(M1).subs(gamma,gamma_of_gas))
  f = lambdify(args=[theta,beta,M1],expr=(eqn.subs(gamma,gamma_of_gas)))
  tolerance = 1e-6
  iteration_counter = 0
  iteration_table = {'theta_i+1':[],'beta_i+1':[],'M_i+1':[],'f_i+1':[]}
  if theta_known == False:
    theta_i = intial_theta_guess_rad
    while f(theta_i,intial_beta_guess_rad,intial_mach_guess) > tolerance or  f(theta_i,intial_beta_guess_rad,intial_mach_guess) < -tolerance:
      theta_i = theta_i - f(theta_i,intial_beta_guess_rad,intial_mach_guess)/partial_theta(theta_i,intial_beta_guess_rad,intial_mach_guess)
      iteration_table['theta_i+1'].append(theta_i),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(theta_i,intial_beta_guess_rad,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\theta'),theta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return theta_i
  if beta_known == False:
    beta_i = intial_beta_guess_rad
    while f(intial_theta_guess_rad,beta_i,intial_mach_guess) > tolerance or  f(intial_theta_guess_rad,beta_i,intial_mach_guess) < -tolerance:
      beta_i = beta_i - f(intial_theta_guess_rad,beta_i,intial_mach_guess)/partial_beta(intial_beta_guess_rad,beta_i,intial_mach_guess)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(beta_i),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,beta_i,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\beta'),beta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return beta_i
  if mach_known == False:
    mach_i = intial_mach_guess
    while f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) > tolerance or  f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) < -tolerance:
      mach_i = mach_i - f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)/partial_mach(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(mach_i),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('M_1'),mach_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return mach_i

In [107]:
beta_rad = ThetaBetaMach(1.4,16*np.pi/180,16*np.pi/180,1.7,True,False,True)
beta_degrees = beta_rad*180/np.pi
displayEquations(symbols('\\beta'),beta_degrees)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.279253,0.478091,1.7,-1.217785e-01
1,0.279253,0.575101,1.7,-4.253078e-02
2,0.279253,0.619732,1.7,-1.448415e-02
3,0.279253,0.637308,1.7,-4.572473e-03
4,0.279253,0.643219,1.7,-1.371045e-03
5,0.279253,0.645032,1.7,-4.024080e-04
6,0.279253,0.645567,1.7,-1.172797e-04
7,0.279253,0.645724,1.7,-3.410791e-05
8,0.279253,0.645769,1.7,-9.913238e-06
9,0.279253,0.645782,1.7,-2.880692e-06


<IPython.core.display.Math object>

In [113]:
beta_rad = ThetaBetaMach(1.32,20*np.pi/180,80*np.pi/180,2.1,True,False,True)
beta_degrees = beta_rad*180/np.pi
displayEquations(symbols('\\beta'),beta_degrees)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.349066,1.392587,2.1,-3.773786e-02
1,0.349066,1.389310,2.1,-3.277805e-02
2,0.349066,1.386404,2.1,-2.842127e-02
3,0.349066,1.383838,2.1,-2.460677e-02
4,0.349066,1.381581,2.1,-2.127654e-02
...,...,...,...,...
66,0.349066,1.366749,2.1,-1.637491e-06
67,0.349066,1.366748,2.1,-1.404357e-06
68,0.349066,1.366748,2.1,-1.204415e-06
69,0.349066,1.366748,2.1,-1.032939e-06


<IPython.core.display.Math object>